In [14]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Read file

In [15]:
fp = open('/content/drive/MyDrive/IR/HW3/pubmed_220_train.txt', 'r')

data = fp.readlines()

print(data[1])
tezt = data[1].replace("BACKGROUND\t", "")


BACKGROUND	The emergence of HIV as a chronic condition means that people living with HIV are required to take more responsibility for the self-management of their condition , including making physical , emotional and social adjustments .



Build JSON file

In [16]:
id_number = 0
content_number = 0
content = ""
data_clean = []
data_temp = {}
for i in data:
  
  if i[0] == '#':   #number
    if len(content) != 0:
      data_temp["contents"] = content
      data_clean.append(data_temp)

    id_number = i.replace("###", "").replace("\n", "")
    content_number = 0
    content = ""
    data_temp = {}
    data_temp["id"] = id_number
  else:         #context
    if content_number == 0:
      content_number = 100
      content += i.replace("\n", "").replace("BACKGROUND\t", "")\
                       .replace("OBJECTIVE\t", "")\
                       .replace("METHODS\t", "")\
                       .replace("RESULTS\t", "")\
                       .replace("CONCLUSIONS\t", "")\
                       .replace("'", "")\
                       .replace("',", "")
    else:
      content += " "
      content += i.replace("\n", "").replace("BACKGROUND\t", "")\
                       .replace("OBJECTIVE\t", "")\
                       .replace("METHODS\t", "")\
                       .replace("RESULTS\t", "")\
                       .replace("CONCLUSIONS\t", "")\
                       .replace("'", "")\
                       .replace("',", "")
    

In [17]:
import json

with open('/content/drive/MyDrive/IR/HW3/data_after.json', 'w') as fp_write:
  for i in data_clean:
    json.dump(i, fp_write)
    fp_write.write('\n')
    


Install package

In [18]:
!sudo apt install gcc libsdl2-dev libffi-dev python-dev libomp-dev

!pip install tdl
!pip install lightgbm

Reading package lists... Done
Building dependency tree       
Reading state information... Done
libffi-dev is already the newest version (3.2.1-8).
python-dev is already the newest version (2.7.15~rc1-1).
libomp-dev is already the newest version (5.0.1-1).
gcc is already the newest version (4:7.4.0-1ubuntu2.3).
libsdl2-dev is already the newest version (2.0.8+dfsg1-1ubuntu1.18.04.4).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 22 not upgraded.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [19]:
!pip install pyserini
!pip3 install torch torchvision torchaudio
!pip install faiss
!python -m pyserini.index.lucene \
  --collection JsonCollection \
  --input /content/drive/MyDrive/IR/HW3/ \
  --index /content/drive/MyDrive/IR/HW3/ \
  --generator DefaultLuceneDocumentGenerator \
  --threads 1 \
  --storePositions --storeDocvectors --storeRaw

!python -m pyserini.search.lucene \
  --index /content/drive/MyDrive/IR/HW3/ \
  --topics /content/drive/MyDrive/IR/HW3/ \
  --output run.sample.txt \
  --bm25



Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
2022-10-18 16:27:15,263 INFO  [main] index.IndexCollection (IndexCollection.java:248) - Setting log level to INFO
2022-10-18 16:27:15,267 INFO  [main] index.IndexCollection (IndexCollection.java:251) - Starting indexer...
2022-10-18 16:27:15,267 INFO  [main] index.IndexCollection (IndexCollection.java:252) - ============ Loading Parameters ============
2022-10-18 16:27:15,268 INFO  [main] index.IndexCollection (IndexCollection.java:253) - DocumentCollection path: /content/drive/MyDrive/IR/HW3/
2022-10-18 16:27:15,271 INFO  [main] index.IndexCollection (IndexCollection.java:254) - CollectionClass: JsonCollection
2022-10-18 16:27:15,271 INFO  [main] index.IndexCollection (IndexCollection.java:255

用反向索引搜尋

In [20]:
from pyserini.search.lucene import LuceneSearcher
import time

start = time.time() #start time

searcher = LuceneSearcher('/content/drive/MyDrive/IR/HW3/')
searcher.set_bm25()
hits = searcher.search('cancer', 20)

for i in range(len(hits)):
    print(f'{i+1:2} {hits[i].docid:4} {hits[i].score:.5f}')
    print(searcher.doc(hits[i].docid).raw())

end = time.time() #end time
invert_search_time = end - start

 1 24747090 2.50920
{
  "id" : "24747090",
  "contents" : "We examine the role of body mass index in the assessment of prostate cancer risk . A total of 3,258 participants who underwent biopsy ( including 1,902 men with a diagnosis of prostate cancer ) were identified from the Selenium and Vitamin E Cancer Prevention Trial . The associations of body mass index with prostate cancer and high grade prostate cancer were examined using logistic regression , adjusting for age , race , body mass index adjusted prostate specific antigen , digital rectal examination , family history of prostate cancer , biopsy history , prostate specific antigen velocity , and time between study entry and the last biopsy . The prediction models were compared with our previously developed body mass index adjusted Prostate Cancer Prevention Trial prostate cancer risk calculator . Of the study subjects 49.1 % were overweight and 29.3 % were obese . After adjustment , among men without a known family history of pro

Linear scan

In [21]:
import json
data_read = []
with open('/content/drive/MyDrive/IR/HW3/data_after.json') as fp_read:
  for i in fp_read:
    #print(type(json.loads(i)))
    data_read.append(json.loads(i))



In [43]:
import time

ans = []

start = time.time() #start time

number = 0
for i in data_read:
  if i['contents'].find('cancer') != -1:
    ans.append(i)

for i in ans:
  number += 1
  print(number)
  print(i['id'])
  print(i['contents'])

end = time.time() #end time
linear_search_time = end - start

Output hidden; open in https://colab.research.google.com to view.

In [45]:
print('反向索引時間：', invert_search_time)
print('線性搜尋時間：', linear_search_time)
print('兩者相差時間：', linear_search_time-invert_search_time)

反向索引時間： 0.1526782512664795
線性搜尋時間： 19.773491382598877
兩者相差時間： 19.620813131332397
